In [ ]:
import requests
import feedparser

# Base api query url
base_url = "http://export.arxiv.org/api/query?"

# Search parameters
query_params = {
    'search_query': "all:rag",
    'start': 0,  # retreive the first 5 results
    'max_results': 5,
    'sortBy': "submittedDate",
    'sortOrder': "descending"
}
query = urllib.parse.urlencode(query_params)

# perform a GET request using the base_url and query
response = requests.get(base_url + query)

# parse the response using feedparser
feed = feedparser.parse(response.text)

# print out feed information
print("Feed title: %s" % feed.feed.title)
print("Feed last updated: %s" % feed.feed.updated)

# print opensearch metadata
print("totalResults for this query: %s" % feed.feed.opensearch_totalresults)
print("itemsPerPage for this query: %s" % feed.feed.opensearch_itemsperpage)
print("startIndex for this query: %s" % feed.feed.opensearch_startindex)

# Run through each entry, and print out information
for entry in feed.entries:
    print("e-print metadata")
    print("arxiv-id: %s" % entry.id.split("/abs/")[-1])
    print("Published: %s" % entry.published)
    print("Title:  %s" % entry.title)

    # Authors
    authors = entry.authors
    if authors:
        print("Authors:  %s" % ",".join(author.name for author in authors))

    # get the links to the abs page and pdf for this e-print
    for link in entry.links:
        if link.rel == "alternate":
            print("abs page link: %s" % link.href)
        elif link.title == "pdf":
            print("pdf link: %s" % link.href)

    # The journal reference, comments and primary_category sections live under
    # the arxiv namespace
    journal_ref = getattr(entry, 'arxiv_journal_ref', 'No journal ref found')
    print("Journal reference: %s" % journal_ref)

    comment = getattr(entry, 'arxiv_comment', 'No comment found')
    print("Comments: %s" % comment)

    primary_category = entry.tags[0]["term"]
    print("Primary Category: %s" % primary_category)

    all_categories = [t["term"] for t in entry.tags]
    print("All Categories: %s" % (", ").join(all_categories))

    print("Abstract: %s" % entry.summary)
    print()


In [65]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Attention is all you need",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

results = client.results(search)

# `results` is a generator; you can iterate over its elements one by one...
for r in client.results(search):
  print(r.title)
# ...or exhaust it into a list. Careful: this is slow for large results sets.
all_results = list(results)
print([r.title for r in all_results])


An Electromagnetism-Inspired Method for Estimating In-Grasp Torque from Visuotactile Sensors
Optimizing OOD Detection in Molecular Graphs: A Novel Approach with Diffusion Models
A new framework of high-order unfitted finite element methods using ALE maps for moving-domain problems
Characterizing the Age of Information with Multiple Coexisting Data Streams
FR-NAS: Forward-and-Reverse Graph Predictor for Efficient Neural Architecture Search
Layer Ensemble Averaging for Improving Memristor-Based Artificial Neural Network Performance
A Dynamic Kernel Prior Model for Unsupervised Blind Image Super-Resolution
Neuromorphic Shack-Hartmann wave normal sensing
Neural Operator induced Gaussian Process framework for probabilistic solution of parametric partial differential equations
DPO: Differential reinforcement learning with application to optimal configuration search
['An Electromagnetism-Inspired Method for Estimating In-Grasp Torque from Visuotactile Sensors', 'Optimizing OOD Detection in Mo

In [31]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
import json


# Load HTML
links = ["https://habr.com/ru/companies/sberbank/articles/805337/",
         "https://habr.com/ru/feed/"]

loader = AsyncHtmlLoader(links)
html = loader.load()

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span","p"])

result = {}
for i in range(len(links)):
    result[links[i]] = docs_transformed[i].page_content
    
f = open("assets/pages_content.json","w")
f.write(json.dumps(result))
f.close()

Fetching pages: 100%|#########################################################################################################| 2/2 [00:00<00:00,  3.36it/s]


## Инициализация модели
Теперь инициализируем модель GigaChat.

In [46]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [47]:
from langchain.chat_models.gigachat import GigaChat
llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

Для проверки спросим у модели вопрос про цвет плаща без какого-либо контекста. Возможно, она и так будет давать ожидаемый ответ...

In [48]:
from langchain.schema import HumanMessage

question = "Какой Loss использует Yolov8?"
llm([HumanMessage(content=question)]).content[0:200]

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'YOLOv8 использует несколько видов потерь для обучения модели. Однако, основной вид потери, который используется в большинстве случаев, это Cross-Entropy Loss (CEL). \n\nCross-Entropy Loss (CEL) - это фу'

Видим, что модель не отвечает так, как нам хотелось бы, поэтому применим RAG-подход.

## Подготовка документа

Для работы с документом нам нужно разделить его на части. Для этого используем `TextLoader` для загрузки книги и `RecursiveCharacterTextSplitter`, чтобы разделить текст на приблизительно равные куски в ~1000 символов с перекрытием в ~200 символов. Этот тип сплиттера сам выбирает каким способом следует оптимально разделять документ (по абзацам, по предложениям и т.д.)

In [50]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("assets/habr_paper.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 20


После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

In [51]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [52]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [53]:
print(f"... {str(docs[0])[620:800]} ...")

... ение (fork) открытой библиотеки LangСhain на Python. Её главная цель — облегчить жизнь разработчику. Библиотека состоит из большого количества различных компонентов, которые позвол ...


## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [54]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [57]:
print(qa_chain)

combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=GigaChat(credentials='YmZlNGMwYWQtM2E0ZS00NzQ3LWIzMzQtZWYxN2NjNTYxODEyOmIyMjkxZTI2LTg4NjktNDc1Yy05NjE5LTg2NzUxNzc3MWZmYg==', scope='GIGACHAT_API_CORP', verify_ssl_certs=False, _client=<gigachat.GigaChat object at 0x7f82e5000940>)), document_variable_name='context') retriever=VectorStoreRetriever(tags=['Chroma', 'GigaChatEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f82e5090a30>)


Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [58]:
qa_chain({"query": question})

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Какой Loss использует Yolov8?',
 'result': 'Я не знаю ответа на этот вопрос.'}

Несколько дополнительных вопросов для проверки работоспособности:

In [67]:
qa_chain({"query": "Сформируй запрос в поисковую систему для поиска нформации про RAG"})

{'query': 'Сформируй запрос в поисковую систему для поиска нформации про RAG',
 'result': 'Извините, но я не могу выполнить ваш запрос. Пожалуйста, уточните свой вопрос или попробуйте еще раз.'}

In [24]:
qa_chain({"query": "Основные особенности Yolov8"})

{'query': 'Основные особенности Yolov8',
 'result': 'YOLOv8 имеет несколько особенностей, которые отличают его от других версий YOLO. Во-первых, он использует новую архитектуру, которая сочетает в себе модули FAN и PAN. Это позволяет ему лучше захватывать особенности на разных масштабах и разрешениях, что важно для точного обнаружения объектов разного размера и формы. Кроме того, YOLOv8 превосходит YOLOv5 по нескольким параметрам, включая более высокую метрику mAP и меньшее количество выбросов при измерении против RF100. Он также превосходит YOLOv5 для каждого RF100 категории.'}